In [1]:
import pyarrow.parquet as pq
import pandas as pd
from pymongo import MongoClient

In [2]:
table = pq.read_table('sdsc_data/output.parquet')
df = table.to_pandas()

In [3]:
df.head()

,news,id,collectiondate,title,url,publishdate,author,keywords,src,language,newsindex
0,"Countering Collins’ claims, Democrats said Tru...",1330233,2019-12-18,Partisan rage envelops Congress on the eve of...,http://www.nydailynews.com/news/politics/ny-...,2019-12-18,"[Dave Goldiner, Chris Sommerfeldt]","[rivals, expected, eve, president, volodymyr, ...",http://www.nydailynews.com/,None,None
1,After Cizikas` goal put the Islanders ahead by...,1330234,2019-12-18,Predators score 7 straight goals to beat Isla...,https://www.nydailynews.com/sports/hockey/is...,2019-12-18,[Allan Kreda],"[york, straight, islanders, goal, goals, sweep...",http://www.nydailynews.com/,None,None
2,"It was a tough call over David Fizdale, but Fi...",1330235,2019-12-18,The Daily News’ Knicks all-decade team for th...,http://www.nydailynews.com/sports/basketball...,2019-12-18,[Stefan Bondy],"[mother, daily, knicks, matt, practice, wins, ...",http://www.nydailynews.com/,None,None
3,And it’s a far cry from tactics such as “stamp...,1330236,2019-12-18,Kindergarten student earns enough money from ...,http://www.nydailynews.com/news/national/ny-...,2019-12-18,[Theresa Braine],"[debt, money, students, pay, meal, childs, thr...",http://www.nydailynews.com/,None,None
4,The Hawkeyes had slogged through 17 consecutiv...,1330237,2019-12-18,"Hayden Fry, Texan who turned around Iowa foot...",https://www.nydailynews.com/sports/football/...,2019-12-18,[Ralph D. Russo],"[hawkeyes, texan, unveiled, familiar, worn, un...",http://www.nydailynews.com/,None,None


In [4]:
client = MongoClient('mongodb://localhost:27017/')
db = client['mydatabase']
collection = db['sdsc-data']
#collection.delete_many({}) # Clears out collection

In [5]:
data = df.to_dict(orient = 'records') # Converts DataFrame to list of dictionaries (JSON-like)

In [6]:
## START OF POPULATING DATA INTO MONGODB

In [7]:
from bson import BSON
from datetime import datetime
 
for doc in data: # Have to preprocess some column to fit datatype for MongoDB
    publish_date = datetime.combine(doc['publishdate'], datetime.min.time())
    collection_date = datetime.combine(doc['collectiondate'], datetime.min.time())
    
    publish_date_timestamp = int(publish_date.timestamp() * 1000)
    collection_date_timestamp = int(collection_date.timestamp() * 1000)
    
    doc['publishdate'] = publish_date_timestamp
    doc['collectiondate'] = collection_date_timestamp
    
    doc['author'] = doc['author'].tolist()
    doc['keywords'] = doc['keywords'].tolist()

In [8]:
collection.insert_many(data)

In [ ]:
## END OF POPULATING DATA INTO MONGODB